In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
df = (pd.read_csv("municipios.csv", index_col = 0)
    .assign(tratamento = lambda _: _.fuso_horario != "America/Sao_Paulo",
            coord = lambda _: list(zip(np.deg2rad(_.latitude), np.deg2rad(_.longitude))))
    .filter(["tratamento", "coord"])
)

In [3]:
df.head()

,tratamento,coord
id_municipio,,
5200050,False,"(-0.2924700587444458, -0.8629117261370205)"
5200100,False,"(-0.28269097894552153, -0.8500748294886021)"
5200134,False,"(-0.30361747667693356, -0.8792078653628914)"
5200159,False,"(-0.28645565414207336, -0.8755566365677194)"
5200175,False,"(-0.2614119247052067, -0.8339584591756866)"


In [4]:
abstencao = (pd.read_csv("abstencao.csv", index_col = 0)
    .assign(abstencao = lambda _: _.abstencoes / _.aptos)
    .drop(["aptos", "abstencoes", "secoes"], axis = 1)
)
abstencao.head()

,id_municipio,ano,turno,abstencao
0,1600303,2002,1,0.140636
1,1600600,2002,1,0.131125
2,1600303,2002,2,0.162194
3,1600600,2002,2,0.158141
4,5200258,2002,1,0.208052


In [5]:
municipios_controle = df.query("tratamento == 0")
municipios_tratamento = df.query("tratamento == 1")

In [6]:
coordenadas = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric="haversine").fit(list(municipios_controle.coord))
distancias, indices = coordenadas.kneighbors(list(municipios_tratamento.coord))
municipios_tratamento = municipios_tratamento.assign(distancia = distancias * 6371)
municipios_tratamento.head()

,tratamento,coord,distancia
id_municipio,,,
2605459,True,"(-0.06701366195957427, -0.5656734278761262)",365.791137
5100102,True,"(-0.26518532654801846, -0.9837234169600679)",412.503233
5100201,True,"(-0.24523621319772324, -0.9103654831694943)",151.365068
5100250,True,"(-0.17220709943822532, -0.9788975815783036)",312.664214
5100300,True,"(-0.30220899597057416, -0.9288310666555943)",2.209545


In [7]:
coordenadas = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric="haversine").fit(list(municipios_tratamento.coord))
distancias, indices = coordenadas.kneighbors(list(municipios_controle.coord))
municipios_controle = municipios_controle.assign(distancia = distancias * 6371)
municipios_controle.head()

,tratamento,coord,distancia
id_municipio,,,
5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.869590
5200100,False,"(-0.28269097894552153, -0.8500748294886021)",317.668256
5200134,False,"(-0.30361747667693356, -0.8792078653628914)",239.108406
5200159,False,"(-0.28645565414207336, -0.8755566365677194)",193.795343
5200175,False,"(-0.2614119247052067, -0.8339584591756866)",352.222984


In [8]:
df.head()

,tratamento,coord
id_municipio,,
5200050,False,"(-0.2924700587444458, -0.8629117261370205)"
5200100,False,"(-0.28269097894552153, -0.8500748294886021)"
5200134,False,"(-0.30361747667693356, -0.8792078653628914)"
5200159,False,"(-0.28645565414207336, -0.8755566365677194)"
5200175,False,"(-0.2614119247052067, -0.8339584591756866)"


In [13]:
base = (
pd.concat([municipios_controle, municipios_tratamento])
#  .query("distancia < 150")
#  .drop("distancia", axis = 1)
 .assign(incluso = 1)
 .reset_index()
.merge(df.reset_index(), how = "right")
.assign(incluso = lambda _: np.where(_.incluso.isna(), 0, 1))
)
base.to_csv("base.csv")
base.head()

,id_municipio,tratamento,coord,distancia,incluso
0,5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.869590,1
1,5200100,False,"(-0.28269097894552153, -0.8500748294886021)",317.668256,1
2,5200134,False,"(-0.30361747667693356, -0.8792078653628914)",239.108406,1
3,5200159,False,"(-0.28645565414207336, -0.8755566365677194)",193.795343,1
4,5200175,False,"(-0.2614119247052067, -0.8339584591756866)",352.222984,1


In [21]:
abst = pd.merge(base, abstencao).merge(pd.read_csv("populacao.csv", index_col= 0))
abst.to_csv("base_abstencao.csv")
abst.head()

,id_municipio,tratamento,coord,distancia,incluso,ano,turno,abstencao,populacao
0,5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.86959,1,2002,1,0.136529,5415
1,5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.86959,1,2002,2,0.199327,5415
2,5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.86959,1,2006,1,0.161905,6531
3,5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.86959,1,2006,2,0.203101,6531
4,5200050,False,"(-0.2924700587444458, -0.8629117261370205)",279.86959,1,2010,1,0.161510,6876
